In [2]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Configuration
num_records = 50
users = [f"user_{i}" for i in range(1, 21)]
devices = ['desktop', 'mobile', 'tablet']
short_words = ['LOG', 'ACT', 'WEB', 'APP', 'USR', 'NET', 'DAT', 'SFX']
start_date = datetime(2025, 5, 1)

# Generate synthetic session log data with custom session_id
data = []
for _ in range(num_records):
    session_start = start_date + timedelta(
        days=random.randint(0, 30),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59)
    )
    session_length = timedelta(minutes=random.randint(5, 120))
    session_end = session_start + session_length

    session_id = f"{random.randint(0, 999):03d}-{random.choice(short_words)}"

    data.append({
        'session_id': session_id,
        'user_id': random.choice(users),
        'device_type': random.choice(devices),
        'start_time': session_start.isoformat(),
        'end_time': session_end.isoformat(),
        
    })

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('user_sessions.csv', index=False)

# Preview
print(df.head())


  session_id  user_id device_type           start_time             end_time
0    760-DAT  user_19      tablet  2025-05-13T15:50:00  2025-05-13T16:54:00
1    918-USR  user_13     desktop  2025-05-23T08:09:00  2025-05-23T08:30:00
2    860-SFX   user_1      tablet  2025-05-24T21:49:00  2025-05-24T22:30:00
3    857-DAT   user_8      tablet  2025-05-07T11:35:00  2025-05-07T12:43:00
4    015-USR   user_6      tablet  2025-05-06T10:53:00  2025-05-06T11:36:00


In [3]:
# SECTION 2: Full Extraction
df_full = pd.read_csv("user_sessions.csv", parse_dates=["start_time", "end_time"])
print(f"Extracted {len(df_full)} rows fully.")
print(df_full.head())


Extracted 50 rows fully.
  session_id  user_id device_type          start_time            end_time
0    760-DAT  user_19      tablet 2025-05-13 15:50:00 2025-05-13 16:54:00
1    918-USR  user_13     desktop 2025-05-23 08:09:00 2025-05-23 08:30:00
2    860-SFX   user_1      tablet 2025-05-24 21:49:00 2025-05-24 22:30:00
3    857-DAT   user_8      tablet 2025-05-07 11:35:00 2025-05-07 12:43:00
4    015-USR   user_6      tablet 2025-05-06 10:53:00 2025-05-06 11:36:00


In [4]:
# SECTION 3: Simulate Initial Last Extraction Time
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-15 12:00:00")



In [5]:
# SECTION 4: Incremental Extraction
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()

df = pd.read_csv("user_sessions.csv", parse_dates=["start_time", "end_time"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['start_time'] > last_extraction_time]

print(f"Extracted {len(df_incremental)} rows incrementally since {last_extraction}.")
print(df_incremental.head())


Extracted 50 rows incrementally since 2025-04-15 12:00:00.
  session_id  user_id device_type          start_time            end_time
0    760-DAT  user_19      tablet 2025-05-13 15:50:00 2025-05-13 16:54:00
1    918-USR  user_13     desktop 2025-05-23 08:09:00 2025-05-23 08:30:00
2    860-SFX   user_1      tablet 2025-05-24 21:49:00 2025-05-24 22:30:00
3    857-DAT   user_8      tablet 2025-05-07 11:35:00 2025-05-07 12:43:00
4    015-USR   user_6      tablet 2025-05-06 10:53:00 2025-05-06 11:36:00


In [6]:
# SECTION 5: Save New Timestamp
new_checkpoint = df['start_time'].max()
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())

print(f"Updated last_extraction.txt to {new_checkpoint}")


Updated last_extraction.txt to 2025-05-31 07:31:00


In [ ]:

# SECTION 6: Transform Full Data

# Reload full dataset in case it's not in memory
df_full = pd.read_csv("user_sessions.csv", parse_dates=["start_time", "end_time"])

# 1. CLEANING: Remove duplicates
df_full.drop_duplicates(inplace=True)

# 2. ENRICHMENT: Add session duration in minutes
df_full["session_duration_minutes"] = (df_full["end_time"] - df_full["start_time"]).dt.total_seconds() / 60

# 3. STRUCTURAL: Standardize user_id to uppercase
df_full["user_id"] = df_full["user_id"].str.upper()

# Save transformed full data
df_full.to_csv("transformed_full.csv", index=False)

print(f"✅ Transformed full dataset with {len(df_full)} records.")
print(df_full.head())




--- SECTION 6: Transform Full Data ---
✅ Transformed full dataset with 50 records.
  session_id  user_id device_type          start_time            end_time  \
0    760-DAT  USER_19      tablet 2025-05-13 15:50:00 2025-05-13 16:54:00   
1    918-USR  USER_13     desktop 2025-05-23 08:09:00 2025-05-23 08:30:00   
2    860-SFX   USER_1      tablet 2025-05-24 21:49:00 2025-05-24 22:30:00   
3    857-DAT   USER_8      tablet 2025-05-07 11:35:00 2025-05-07 12:43:00   
4    015-USR   USER_6      tablet 2025-05-06 10:53:00 2025-05-06 11:36:00   

   session_duration_minutes  
0                      64.0  
1                      21.0  
2                      41.0  
3                      68.0  
4                      43.0  


In [ ]:
# SECTION 7: Transform Incremental Data

# Ensure df_incremental exists — if not, re-create it
try:
    df_incremental
except NameError:
    df = pd.read_csv("user_sessions.csv", parse_dates=["start_time", "end_time"])
    with open("last_extraction.txt", "r") as f:
        last_extraction = f.read().strip()
    last_extraction_time = pd.to_datetime(last_extraction)
    df_incremental = df[df["start_time"] > last_extraction_time]

# 1. CLEANING: Remove duplicates
df_incremental.drop_duplicates(inplace=True)

# 2. ENRICHMENT: Add session duration
df_incremental["session_duration_minutes"] = (df_incremental["end_time"] - df_incremental["start_time"]).dt.total_seconds() / 60

# 3. STRUCTURAL: Standardize user_id
df_incremental["user_id"] = df_incremental["user_id"].str.upper()

# Save transformed incremental data
df_incremental.to_csv("transformed_incremental.csv", index=False)

print(f"✅ Transformed incremental dataset with {len(df_incremental)} records.")
print(df_incremental.head())



--- SECTION 7: Transform Incremental Data ---
✅ Transformed incremental dataset with 50 records.
  session_id  user_id device_type          start_time            end_time  \
0    760-DAT  USER_19      tablet 2025-05-13 15:50:00 2025-05-13 16:54:00   
1    918-USR  USER_13     desktop 2025-05-23 08:09:00 2025-05-23 08:30:00   
2    860-SFX   USER_1      tablet 2025-05-24 21:49:00 2025-05-24 22:30:00   
3    857-DAT   USER_8      tablet 2025-05-07 11:35:00 2025-05-07 12:43:00   
4    015-USR   USER_6      tablet 2025-05-06 10:53:00 2025-05-06 11:36:00   

   session_duration_minutes  
0                      64.0  
1                      21.0  
2                      41.0  
3                      68.0  
4                      43.0  
